In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch # type: ignore
import sys; sys.path.append('../../')
from  giagrad.tensor import Tensor
import numpy as np
from giagrad.display import draw_dot
import torch.nn as nn
import string
import math
from itertools import chain
from numpy.lib.stride_tricks import as_strided
np.random.seed(1234)

# Conv 3D

## Helpers

In [22]:
import string
from math import floor
alphabet = [chr(i) for i in chain(range(33,127), range(161, 1500))]
# Channels out, Channels in, kernel Height, kernel Width
def kernel(Cout, Cin, kD, kH, kW, a=0, b=4):
    return np.random.randint(a, b, (Cout, Cin, kD, kH, kW))

def dataBatched(N, Cin, Din, Hin, Win):
    return np.array(alphabet[:N*Cin*Din*Hin*Win], dtype=object).reshape((N, Cin, Din, Hin, Win))
#     return np.random.randint(-1000, 1000, (N, Cin, Hin, Win, Din))

def dataUnBatched(Cin, Din, Hin, Win):
    return np.array(alphabet[:Cin*Hin*Win*Din], dtype=object).reshape((Cin, Din, Hin, Win))

## Forward

Para la convolucion se necesita convertir con as_strided el array anterior para que tenga dimension:
    $$(N, D_{in}, C_{in}, H_{in}, W_{in})$$
Donde 
$$
D_{out} = \left\lfloor 
            \frac{D_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[0] -1) - 1}
                 {stride[0]} + 1 
           \right\rfloor \\
H_{out} = \left\lfloor 
            \frac{H_{in} + 2 \times padding[1] - dilation[1] \times (kernel\_size[1] -1) - 1}
                 {stride[1]} + 1
           \right\rfloor \\
W_{out} = \left\lfloor 
            \frac{W_{in} + 2 \times padding[2] - dilation[2] \times (kernel\_size[2] -1) - 1}
                 {stride[2]} + 1 
           \right\rfloor \\
$$

In [23]:
def out_shape(N, Cout, in_, padding, stride, dilation, kernel_size):
    in_ = np.array(in_, dtype=np.float32)
    padding = np.array(padding, dtype=np.float32)
    stride = np.array(stride, dtype=np.float32)
    dilation = np.array(dilation, dtype=np.float32)
    kernel_size = np.array(kernel_size, dtype=np.float32)
    out = (in_ + 2*padding - dilation*(kernel_size-1) -1) / stride + 1
    return np.insert(out, 0, (N, Cout)).astype(int)

In [30]:
N, Cin, Din, Hin, Win = 1, 1, 2, 5, 5
kD, kH, kW = 2, 2, 2
Cout = 3
padding = (0, 0, 0)
dilation = (1, 2, 1)
stride = (1, 1, 1)
# define output shape
N, Cout, Dout, Hout, Wout = out_shape(
          N, 
          Cout, 
          in_=(Din, Hin, Win),
          padding=padding,
          stride=stride,
          dilation=dilation,
          kernel_size=(kD, kH, kW)   
      )
shape_out = (
#             N, 
             Cout, Dout, Hout, Wout)
print('out_shape: ', shape_out)

out_shape:  (3, 1, 3, 4)


In [31]:
# DATA
data = dataUnBatched(Cin, Din, Hin, Win)
# random KERNEL
k = kernel(Cout, Cin, kD, kH, kW)

tenemos:

    - [ -8, -7, -6 , -5 , -4 , -3 , -2 , -1 ] * itemsize in bytes
    
    posicion -1: dilation[2], horizontal dilation
    posicion -2: Win * dilation[1], vertical dilation
    posicion -3: Hin * Win * dilation[0], depth dilation
    posicion -4: Hin * Win * Din, jump to next Channel
    posicion -5: stride[2], horizontal stride
    posicion -6: Win * stride[1], vertical stride
    posicion -7: Hin * Win * stride[0], depth stride
    posicion -8: Hin*Win*Cin*Din, next observation in Batched data

y el shape del strided array tiene que ser:
    
    -  [ -8, -7, -6 , -5 , -4 , -3 , -2 , -1 ]
    
    posicion -1: kW
    posicion -2: kH
    posicion -3: kD
    posicion -4: Cin
    posicion -5: Wout
    posicion -6: Hout
    posicion -7: Dout
    posicion -8: N

In [32]:
stridedShape = (
#     N,
    Dout,
    Hout, 
    Wout, 
    Cin,
    kD,
    kH, 
    kW
)

strides = np.array([
#      Hin*Win*Din*Cin,
     Hin*Win*stride[0],
     Win*stride[1],
     stride[2],
     Hin*Win*Din,
     Hin*Win*dilation[0],
     Win*dilation[1],
     dilation[2]]) * data.itemsize

print('strides no byte: ', strides/data.itemsize)
print('strides in bytes: ', strides)
print('strided output shape: ', stridedShape)
print('kernel shape: ', k.shape)
print('data shape: ', data.shape)
print('desired convolutional shape:', shape_out)

strides no byte:  [25.  5.  1. 50. 25. 10.  1.]
strides in bytes:  [200  40   8 400 200  80   8]
strided output shape:  (1, 3, 4, 1, 2, 2, 2)
kernel shape:  (3, 1, 2, 2, 2)
data shape:  (1, 2, 5, 5)
desired convolutional shape: (3, 1, 3, 4)


In [33]:
# expand data to apply tensordot
strided_array = as_strided(data, shape=stridedShape, strides=strides)

In [37]:
convolved = np.swapaxes(np.tensordot(k, strided_array, axes=([1, 2, 3, 4], [3, 4, 5, 6, ])), 0, 1)
print('convolved shape: ', convolved.shape)
print('desired shape: ', shape_out)

convolved shape:  (1, 3, 3, 4)
desired shape:  (3, 1, 3, 4)


## Test MyGrad

```python
class ConvND:
    def __call__(self, x, w, *, stride, padding=0, dilation=1):
        # x ... data:    (N, C, X0, X1, ...)
        # w ... filters: (F, C, W0, W1, ...)
```

In [11]:
from mygrad import ConvND

In [12]:
conv_layer = ConvND()
mygrad_convolve = conv_layer(data, k, stride=stride, padding=padding, dilation=dilation)
print('mygrad convolved shape: ', mygrad_convolve.shape)
print('desired shape: ', shape_out)

mygrad convolved shape:  (2, 3, 4, 3, 4)
desired shape:  (2, 3, 4, 3, 4)


In [13]:
# CHECK!!!
np.all(convolved == mygrad_convolve)

True